In [40]:
pip install pandas

In [41]:
import numpy

In [42]:
import pandas

In [43]:
train_path = "Genre classification dataset 1\\Genre Classification Dataset\\train_data.txt"
train_data = pandas.read_csv(train_path, sep=':::', names=['Title', 'Genre', 'Description'], engine='python')
train_data

test_path = "Genre classification dataset 1\\Genre Classification Dataset\\test_data.txt"
test_data = pandas.read_csv(test_path, sep=':::', names=['Id', 'Title', 'Description'], engine='python')
test_data

,Id,Title,Description
0,1,Edgar's Lunch (1998),"L.R. Brane loves his life - his car, his apar..."
1,2,La guerra de papá (1977),"Spain, March 1964: Quico is a very naughty ch..."
2,3,Off the Beaten Track (2010),One year in the life of Albin and his family ...
3,4,Meu Amigo Hindu (2015),"His father has died, he hasn't spoken with hi..."
4,5,Er nu zhai (1955),Before he was known internationally as a mart...
...,...,...,...
54195,54196,"""Tales of Light & Dark"" (2013)","Covering multiple genres, Tales of Light & Da..."
54196,54197,Der letzte Mohikaner (1965),As Alice and Cora Munro attempt to find their...
54197,54198,Oliver Twink (2007),A movie 169 years in the making. Oliver Twist...
54198,54199,Slipstream (1973),"Popular, but mysterious rock D.J Mike Mallard..."


In [44]:
print(train_data)

                                              Title          Genre  \
1                     Oscar et la dame rose (2009)          drama    
2                                     Cupid (1997)       thriller    
3                 Young, Wild and Wonderful (1980)          adult    
4                            The Secret Sin (1915)          drama    
5                           The Unrecovered (2007)          drama    
...                                             ...            ...   
54210                              "Bonino" (1953)         comedy    
54211                  Dead Girls Don't Cry (????)         horror    
54212    Ronald Goedemondt: Ze bestaan echt (2008)    documentary    
54213                     Make Your Own Bed (1944)         comedy    
54214   Nature's Fury: Storm of the Century (2006)        history    

                                             Description  
1       Listening in to a conversation between his do...  
2       A brother and sister with a past 

In [45]:


test_solution_path = "Genre classification dataset 1\\Genre Classification Dataset\\test_data_solution.txt"
test_solution_data = pandas.read_csv(test_solution_path, sep=':::', names=['Id', 'Title','Genre', 'Description'], engine='python')
test_solution_data

,Id,Title,Genre,Description
0,1,Edgar's Lunch (1998),thriller,"L.R. Brane loves his life - his car, his apar..."
1,2,La guerra de papá (1977),comedy,"Spain, March 1964: Quico is a very naughty ch..."
2,3,Off the Beaten Track (2010),documentary,One year in the life of Albin and his family ...
3,4,Meu Amigo Hindu (2015),drama,"His father has died, he hasn't spoken with hi..."
4,5,Er nu zhai (1955),drama,Before he was known internationally as a mart...
...,...,...,...,...
54195,54196,"""Tales of Light & Dark"" (2013)",horror,"Covering multiple genres, Tales of Light & Da..."
54196,54197,Der letzte Mohikaner (1965),western,As Alice and Cora Munro attempt to find their...
54197,54198,Oliver Twink (2007),adult,A movie 169 years in the making. Oliver Twist...
54198,54199,Slipstream (1973),drama,"Popular, but mysterious rock D.J Mike Mallard..."


In [46]:
pip install scikit-learn

In [50]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# Example data
train_data = pd.DataFrame({
    'ID': range(1, 101),
    'title': [f'Title{i}' for i in range(1, 101)],
    'Genre': ['Action', 'Comedy', 'Drama'] * 33 + ['Action'],
    'Description': [
        'Action movie with lots of action scenes', 
        'Comedy movie with lots of laughs', 
        'Dramatic movie with emotional scenes'
    ] * 33 + ['Action movie with a hero']
})

test_solution_data = pd.DataFrame({
    'ID': range(101, 121),
    'title': [f'Title{i}' for i in range(101, 121)],
    'Genre': ['Action', 'Comedy', 'Drama'] * 6 + ['Action', 'Comedy'],
    'Description': [
        'A thrilling action packed movie', 
        'A hilarious comedy', 
        'A deep and thoughtful drama'
    ] * 6 + ['An action adventure', 'A comedy show']
})

# Drop the 'ID' and 'title' columns as they are not needed
train_data = train_data.drop(columns=['ID', 'title'])
test_solution_data = test_solution_data.drop(columns=['ID', 'title'])

# TF-IDF Vectorization using unigrams and bigrams
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=1000)
X_train_tfidf = vectorizer.fit_transform(train_data['Description'])
X_test_tfidf = vectorizer.transform(test_solution_data['Description'])

# Label Encoding the 'Genre' column
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_data['Genre'])
y_test = label_encoder.transform(test_solution_data['Genre'])

# Select K Best features (TF-IDF has already limited to 1000 features)
selector = SelectKBest(chi2, k=1000)
X_train_selected = selector.fit_transform(X_train_tfidf, y_train)
X_test_selected = selector.transform(X_test_tfidf)

# Apply standard scaler
scaler = StandardScaler(with_mean=False)  # TF-IDF output is sparse matrix; with_mean=False to avoid error
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

# Define the SGD classifier
sgd = SGDClassifier(loss='log_loss')

# Define hyperparameters for tuning
param_grid = {
    'alpha': [0.0001, 0.001, 0.01, 0.1],
    'penalty': ['l2', 'l1', 'elasticnet'],
    'max_iter': [1000, 2000, 3000],
    'tol': [1e-3, 1e-4, 1e-5]
}

# Set up GridSearchCV with 5 splits
grid_search = GridSearchCV(sgd, param_grid, scoring='accuracy', cv=5, n_jobs=-1)

# Fit the model
grid_search.fit(X_train_scaled, y_train)

# Get the best model
best_sgd = grid_search.best_estimator_

# Predict on the test set
y_pred = best_sgd.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

print("Best Hyperparameters: ", grid_search.best_params_)
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("Confusion Matrix: \n", conf_matrix)


c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:776: UserWarning: k=1000 is greater than n_features=23. All the features will be returned.
  warnings.warn(
c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Best Hyperparameters:  {'alpha': 0.0001, 'max_iter': 1000, 'penalty': 'l2', 'tol': 0.001}
Accuracy:  0.7
Precision:  0.5384615384615385
Recall:  0.7
Confusion Matrix: 
 [[7 0 0]
 [0 7 0]
 [6 0 0]]
